In [37]:
# Reading and Analyzing Bicing Data from Parquet
#
# This script demonstrates how to read and analyze the Bicing data stored in Parquet format.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set plot style
sns.set_palette('husl')

# 1. Reading the Parquet File
print("1. Reading the Parquet File\n")

# Read the Parquet file
file_path = './compressed_data/all_data.parquet'
df = pd.read_parquet(file_path)

# Display basic information about the dataset
print("Dataset Info:")
df.info()

print("\nFirst few rows:")
print(df.head())

# 2. Reading Specific Columns
print("\n2. Reading Specific Columns\n")

# Read only specific columns
df_subset = pd.read_parquet(
    file_path,
    columns=['timestamp', 'data']
)
print("Subset of data (first few rows):")
print(df_subset.head())

# 3. Filtering Data
print("\n3. Filtering Data\n")

# Read data for a specific date range
start_date = datetime(2023, 9, 5)
df_filtered = pd.read_parquet(
    file_path,
    filters=[("timestamp", ">", start_date)]
)

print(f"Number of records after {start_date}: {len(df_filtered)}")

# 4. Time Series Analysis
print("\n4. Time Series Analysis\n")

# Group by hour and count records
hourly_counts = df['timestamp'].dt.floor('H').value_counts().sort_index()

# Plot
plt.figure(figsize=(15, 6))
hourly_counts.plot(kind='line')
plt.title('Number of Records per Hour')
plt.xlabel('Hour')
plt.ylabel('Number of Records')
plt.grid(True)
plt.savefig('./compressed_data/hourly_records.png')
plt.close()

print("Time series plot saved as 'analysis/hourly_records.png'")

# 5. Memory Efficiency
print("\n5. Memory Efficiency\n")

def get_size_df(df):
    return df.memory_usage(deep=True).sum() / 1024 / 1024

# Full dataset
df_full = pd.read_parquet(file_path)
print(f"Full dataset size: {get_size_df(df_full):.2f} MB")

# Only specific columns
df_cols = pd.read_parquet(file_path, columns=['timestamp', 'ttl'])
print(f"Selected columns size: {get_size_df(df_cols):.2f} MB")

# Filtered dataset
df_filtered = pd.read_parquet(
    file_path,
    filters=[("timestamp", ">", datetime(2023, 9, 6))]
)
print(f"Filtered dataset size: {get_size_df(df_filtered):.2f} MB") 

1. Reading the Parquet File

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10078 entries, 0 to 10077
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ttl           10078 non-null  int64         
 1   last_updated  10078 non-null  int64         
 2   data          10078 non-null  object        
 3   timestamp     10078 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 315.1+ KB

First few rows:
   ttl  last_updated                                               data  \
0   -1    1694063435  {'stations': [{'is_charging_station': True, 'i...   
1   -1    1694083661  {'stations': [{'is_charging_station': True, 'i...   
2   -1    1694116657  {'stations': [{'is_charging_station': True, 'i...   
3   -1    1694086176  {'stations': [{'is_charging_station': True, 'i...   
4   -1    1694068296  {'stations': [{'is_charging_station': True, 'i...   

          

/var/folders/12/n2x_v3c9753_89hzt6068x340000gn/T/ipykernel_2879/3684005413.py:54: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_counts = df['timestamp'].dt.floor('H').value_counts().sort_index()


Full dataset size: 2.08 MB
Selected columns size: 0.15 MB
Filtered dataset size: 0.62 MB


In [38]:
# Verify that the number of records is the same and the space saved
import os

folder = '../snapshots'
target_format = 'json'

def list_folders(folder):
    """
    List all subdirectories in the specified folder.
    :param folder: Path to the main folder.
    :return: List of folder names.
    """
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

def list_files(folder):
    """
    List all files in the specified directory.
    :param folder: Path to the directory.
    :return: List of file names.
    """
    return [name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))]

def list_all_files(main_folder, dates):
    """
    List all files within subdirectories specified by dates under the main folder.
    :param main_folder: Path to the main folder.
    :param dates: List of subdirectory names to search within.
    :return: List of file paths.
    """
    files = []
    for date in dates:
        folder = os.path.join(main_folder, date)
        for file in list_files(folder):
            files.append(os.path.join(folder, file))
    return files

def get_total_size_mb(folder, dates):
    """
    Calculate total size of all files in the specified folders in MB.
    :param folder: Path to the main folder.
    :param dates: List of subdirectory names to search within.
    :return: Total size in MB.
    """
    total_bytes = 0
    for file_path in list_all_files(folder, dates):
        total_bytes += os.path.getsize(file_path)
    return total_bytes / (1024 * 1024)  # Convert bytes to MB

In [56]:
df_full = pd.read_parquet(file_path)

print(f"Number of records in parquet dataset: {len(df_full)}")
print(f"Number of records in JSON files: {len(list_all_files(folder,list_folders(folder)))}")
print(f"Records difference: {len(df_full)-len(list_all_files(folder,list_folders(folder)))}")
print("---------------------------------------------------------")
print(f"Pandas memmory dataset size: {get_size_df(df_full):.2f} MB")

file_path = './compressed_data/all_data.parquet'
if os.path.isfile(file_path):
    compressed_size = os.path.getsize(file_path) / (1024 * 1024)
    print(f"Parquet file size: {compressed_size:.2f} MB")
else:
    print("Parquet file not found")

json_size = get_total_size_mb(folder, list_folders(folder))
print(f"Size of JSON files: {json_size:.2f} MB")

compression_ratio = json_size / compressed_size
print(f"Compression ratio (JSON/Parquet): {compression_ratio:.2f}x")



Number of records in parquet dataset: 10078
Number of records in JSON files: 10078
Records difference: 0
---------------------------------------------------------
Pandas memmory dataset size: 2.08 MB
Parquet file size: 31.11 MB
Size of JSON files: 1668.64 MB
Compression ratio (JSON/Parquet): 53.64x


In [35]:
# Read a specific station data
station_id = 10
df_full = pd.read_parquet(file_path)
stations_df = pd.DataFrame([row['stations'][station_id-1] for row in df_full['data']])
stations_df.head()

,is_charging_station,is_installed,is_renting,is_returning,last_reported,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,station_id,status,traffic
0,True,1,1,1,1694063319,22,"{'ebike': 3, 'mechanical': 19}",0,21,0,10,IN_SERVICE,None
1,True,1,1,1,1694083623,29,"{'ebike': 9, 'mechanical': 20}",0,14,0,10,IN_SERVICE,None
2,True,1,1,1,1694116639,20,"{'ebike': 1, 'mechanical': 19}",0,23,0,10,IN_SERVICE,None
3,True,1,1,1,1694086034,28,"{'ebike': 8, 'mechanical': 20}",0,15,0,10,IN_SERVICE,None
4,True,1,1,1,1694068096,28,"{'ebike': 8, 'mechanical': 20}",0,15,0,10,IN_SERVICE,None


In [61]:
current_dataset_size = ((1668.64/7)*365/1024)
compresed_dataset_size = current_dataset_size/53.64

print(f"Estimated yearly dataset size: {current_dataset_size:.2f} GB")
print(f"Estimated yearly compressed dataset size: {compresed_dataset_size:.2f} GB")

Estimated yearly dataset size: 84.97 GB
Estimated yearly compressed dataset size: 1.58 GB
